In [2]:
import os
import sys
import time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from preprocess_utility import *
from data_manager import *
from metadata import *

Setting environment for Gordon


No vtk


In [3]:
exclude_nodes = [33, 31]

# Generate score volumes

In [4]:
setting = 2

In [5]:
stack = 'MD635'
first_sec, last_sec = metadata_cache['section_limits'][stack]

In [15]:
t = time.time()
sys.stderr.write('running classifier ...')

run_distributed4(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d %(setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning', 'apply_classifiers_v3.py'),
                'stack': stack,
                'setting': setting},
                kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                exclude_nodes=exclude_nodes,
                argument_type='partition')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 220 seconds = 4 mins

running classifier ...

['gcn-20-33.sdsc.edu', 'gcn-20-31.sdsc.edu'] are excluded
Using nodes: ['gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 124.911462 seconds


In [17]:
t = time.time()
sys.stderr.write('interpolating scoremaps ...')

run_distributed4(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d %(setting)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning', 'interpolate_scoremaps_v3.py'),
                'stack': stack,
                'setting': setting},
                kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                exclude_nodes=exclude_nodes,
                argument_type='partition')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 50s/section; 2440 seconds =  40min

interpolating scoremaps ...

['gcn-20-33.sdsc.edu', 'gcn-20-31.sdsc.edu'] are excluded
Using nodes: ['gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 2461.551323 seconds


In [ ]:
t = time.time()
sys.stderr.write('visualize scoremaps ...')

add_annotation = False

run_distributed4(command='%(script_path)s %(stack)s %(setting)d -b %%(first_sec)d -e %%(last_sec)d %(add_annotation)s' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning', 'visualize_scoremaps_v3.py'),
                 'stack': stack,
                 'setting': setting,
                 'add_annotation': '-a' if add_annotation else ''},
                kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                exclude_nodes=exclude_nodes,
                argument_type='partition')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 900 seconds / stack

visualize scoremaps ...

['gcn-20-33.sdsc.edu', 'gcn-20-31.sdsc.edu'] are excluded


In [ ]:
# for stack in all_stacks:
for stack in ['MD635']:
        
    first_sec, last_sec = metadata_cache['section_limits'][stack]

#     #################################

    t = time.time()
    sys.stderr.write('running svm classifier ...')


    run_distributed4(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d %(train_sample_scheme)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning') + '/apply_classifiers.py',
                    'stack': stack,
                    'train_sample_scheme': train_sample_scheme},
                    kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                    exclude_nodes=exclude_nodes,
                    argument_type='partition')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 220 seconds

# #     #################################

    t = time.time()
    sys.stderr.write('interpolating scoremaps ...')

    run_distributed4(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d %(train_sample_scheme)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning') + '/interpolate_scoremaps_v2.py',
                    'stack': stack,
                    'train_sample_scheme': train_sample_scheme},
                    kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                    exclude_nodes=exclude_nodes,
                    argument_type='partition')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 50s/section

    #################################

    t = time.time()
    sys.stderr.write('visualize scoremaps ...')

    add_annotation = False
    # first_sec, last_sec = DataManager.load_cropbox(stack)[4:]

    run_distributed4(command='%(script_path)s %(stack)s -b %%(first_sec)d -e %%(last_sec)d %(add_annotation)s -t %(train_sample_scheme)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning') + '/visualize_scoremaps_v2.py',
                    'stack': stack,
                     'train_sample_scheme': train_sample_scheme,
                    'add_annotation': '-a' if add_annotation else ''},
                    kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                    exclude_nodes=exclude_nodes,
                    argument_type='partition')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 800 seconds / stack


    #################################

    t = time.time()
    sys.stderr.write('constructing score volumes ...')

    run_distributed4(command='%(script_path)s %(stack)s %%(label)s %(train_sample_scheme)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'reconstruct') + '/construct_score_volume_v2.py',
                    'stack': stack,
                    'train_sample_scheme': train_sample_scheme},
                    kwargs_list=dict(label=structures),
                    exclude_nodes=exclude_nodes,
                    argument_type='single')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 600 seconds

    #################################

    downscale_factor = 32

    #################################

    print stack

    volume_allLabels = {}

    for name_u in structures:
        try:
            volume = DataManager.load_score_volume(stack, label=name_u, downscale=downscale_factor, train_sample_scheme=train_sample_scheme)
            volume_allLabels[name_u] = volume            
#         del volume
        except:
            sys.stderr.write('Score volume for %s does not exist.\n' % name_u)

    t1 = time.time()

    gradient_dir = create_if_not_exists(os.path.join(VOLUME_ROOTDIR, stack, 'score_volume_gradients'))

#     for name_u in structures:
    for name_u in volume_allLabels.keys():

        t = time.time()

        gy_gx_gz = np.gradient(volume_allLabels[name_u].astype(np.float32), 3, 3, 3) 
        # 3.3 second - re-computing is much faster than loading
        # .astype(np.float32) is important; 
        # Otherwise the score volume is type np.float16, np.gradient requires np.float32 and will have to convert which is very slow
        # 20s (float32) vs. 2s (float16)

        sys.stderr.write('Gradient %s: %f seconds\n' % (name_u, time.time() - t))

        t = time.time()

        bp.pack_ndarray_file(gy_gx_gz[0], os.path.join(gradient_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_1_gy.bp' % {'stack':stack, 'label':name_u, 'ds': downscale_factor}))
        bp.pack_ndarray_file(gy_gx_gz[1], os.path.join(gradient_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_1_gx.bp' % {'stack':stack, 'label':name_u, 'ds': downscale_factor}))
        bp.pack_ndarray_file(gy_gx_gz[2], os.path.join(gradient_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_1_gz.bp' % {'stack':stack, 'label':name_u, 'ds': downscale_factor}))

        del gy_gx_gz

        sys.stderr.write('save %s: %f seconds\n' % (name_u, time.time() - t))


    sys.stderr.write('overall: %f seconds\n' % (time.time() - t1))

# Global align

In [21]:
# Align

t = time.time()
sys.stderr.write('align all subjects to atlas ...')

run_distributed4(command='%(script_path)s %%(stack)s %(train_sample_scheme)d 1 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/align_subject_brain_to_atlas_v2.py',
                'train_sample_scheme': train_sample_scheme},
#                 kwargs_list=dict(stack=all_stacks),
                 kwargs_list=dict(stack=['MD635']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t))  # 526 seconds

align all subjects to atlas ...

['gcn-20-33.sdsc.edu', 'gcn-20-31.sdsc.edu'] are excluded
Using nodes: ['gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 442.856862 seconds


In [22]:
# Transform

t = time.time()
sys.stderr.write('transform atlas ...')

run_distributed4(command='%(script_path)s %%(stack)s %(train_sample_scheme)d 1 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/transform_brains.py',
                'train_sample_scheme': train_sample_scheme},
#                 kwargs_list=dict(stack=all_stacks),
                 kwargs_list=dict(stack=['MD635']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 310 seconds

transform atlas ...

['gcn-20-33.sdsc.edu', 'gcn-20-31.sdsc.edu'] are excluded
Using nodes: ['gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 275.624859 seconds


In [24]:
# Visualize

t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

run_distributed4(command='%(script_path)s %%(stack)s %(train_sample_scheme)d 1 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/visualize_aligned_brains_v2.py',
                'train_sample_scheme': train_sample_scheme},
                kwargs_list=dict(stack=['MD635']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 625 seconds

visualize aligned atlas overlay ...

['gcn-20-33.sdsc.edu', 'gcn-20-31.sdsc.edu'] are excluded
Using nodes: ['gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 618.753698 seconds


# Local align

In [25]:
# Local align

t = time.time()
sys.stderr.write('LOCAL align ...')

run_distributed4(command='%(script_path)s %%(stack)s %(train_sample_scheme)d 1 2 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/fit_atlas_structure_to_subject_v2.py',
                'train_sample_scheme': train_sample_scheme},
                kwargs_list=dict(stack=['MD635']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 6807 seconds

LOCAL align ...

['gcn-20-33.sdsc.edu', 'gcn-20-31.sdsc.edu'] are excluded
Using nodes: ['gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 4615.594547 seconds


In [27]:
# Transform

t = time.time()
sys.stderr.write('transform atlas ...')

run_distributed4(command='%(script_path)s %%(stack)s %(train_sample_scheme)d 1 2 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/transform_brains_global_to_local.py',
                'train_sample_scheme': train_sample_scheme},
                kwargs_list=dict(stack=['MD635']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 310 seconds

transform atlas ...

['gcn-20-33.sdsc.edu', 'gcn-20-31.sdsc.edu'] are excluded
Using nodes: ['gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 369.619889 seconds


In [28]:
# Visualize

t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

run_distributed4(command='%(script_path)s %%(stack)s %(train_sample_scheme)d 1 2 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/visualize_aligned_brains_v2_local.py',
                'train_sample_scheme': train_sample_scheme},
                kwargs_list=dict(stack=['MD635']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 190 seconds

visualize aligned atlas overlay ...

['gcn-20-33.sdsc.edu', 'gcn-20-31.sdsc.edu'] are excluded
Using nodes: ['gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 647.941902 seconds


In [13]:
# Transform locally transformed volumes back to atlas space

t = time.time()
sys.stderr.write('transform atlas ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s 1 1 2 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/transform_brains_reverse_global.py'},
                kwargs_list=dict(stack=all_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 310 seconds

transform atlas ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 220.844733 seconds


# Snake

In [6]:
train_sample_scheme

8

In [7]:
# Graphcut

t = time.time()
sys.stderr.write('Graphcut ...')

exclude_nodes = [33]

# for stack in all_stacks:
for stack in ['MD635']:

    run_distributed4(command='%(script_path)s %(stack)s %%(name_sided)s %(train_sample_scheme)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/fit_contour_to_scoremap.py',
                    'stack': stack,
                    'train_sample_scheme': train_sample_scheme},
                    kwargs_list=dict(name_sided=[convert_to_left_name(s) for s in paired_structures] + \
                                     [convert_to_right_name(s) for s in paired_structures] + \
                                     [s for s in singular_structures]),
                    exclude_nodes=exclude_nodes,
                    argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

Graphcut ...

['gcn-20-33.sdsc.edu'] are excluded
gcn-20-31.sdsc.edu is down
Using nodes: ['gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 163.411004 seconds
